In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os
base_dir='../../../assets/flower/102-flowers-dataset'
train_dir=os.path.join(base_dir,"train")
test_dir=os.path.join(base_dir,"test")

In [6]:
# dataset augment
data_transform={
    'train':transforms.Compose([transforms.RandomRotation(45),
                                transforms.CenterCrop(224),
                                transforms.RandomVerticalFlip(p=0.5),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.ColorJitter(brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1),
                                transforms.RandomGrayscale(p=0.025),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]),
    'test':transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
}

In [19]:
batch_size=32
train_datasets=datasets.ImageFolder(os.path.join(base_dir,"train"),data_transform["train"])
test_datasets=datasets.ImageFolder(os.path.join(base_dir,"test"),data_transform["test"])
train_dataloader=DataLoader(train_datasets, batch_size=batch_size,shuffle=True)
test_dataloader=DataLoader(test_datasets,batch_size=batch_size,shuffle=True)

train_datasets,test_datasets

(Dataset ImageFolder
     Number of datapoints: 1020
     Root location: ../../../assets/flower/102-flowers-dataset\train
     StandardTransform
 Transform: Compose(
                RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
                CenterCrop(size=(224, 224))
                RandomVerticalFlip(p=0.5)
                RandomHorizontalFlip(p=0.5)
                ColorJitter(brightness=[0.8, 1.2], contrast=[0.9, 1.1], saturation=[0.9, 1.1], hue=[-0.1, 0.1])
                RandomGrayscale(p=0.025)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 Dataset ImageFolder
     Number of datapoints: 6149
     Root location: ../../../assets/flower/102-flowers-dataset\test
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
                CenterCrop(size=(224, 224))
                Normalize(mean=[

In [26]:
model_name="resnet"
feature_extract=True
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [33]:
def set_parameter_requires_grad(model,feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad=False

In [34]:
resnet152_model=models.resnet152(pretrained=True)
set_parameter_requires_grad(resnet152_model,feature_extract)
num_fc=resnet152_model.fc.in_features
resnet152_model.fc=nn.Sequential(nn.Linear(num_fc,102),nn.LogSoftmax(dim=1))
resnet152_model=resnet152_model.to(device)
resnet152_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [36]:
params_to_update=[]
for name,param in resnet152_model.named_parameters():
    if param.requires_grad:
        params_to_update.append(param)
        print(f"{name}\t")

fc.0.weight	
fc.0.bias	


In [37]:
learning_rate=0.01
optimizer=optim.Adam(params_to_update,lr=learning_rate)
scheduler=optim.lr_scheduler.StepLR(optimizer, step_size=7,gamma=0.1)
criterion=nn.NLLLoss()